### API Requests

In [27]:
import os
import json
import base64
import requests

from typing import List, Union
from dotenv import load_dotenv
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from datetime import datetime, timedelta, timezone

# Spark
from pyspark.sql.types import StringType
from delta import configure_spark_with_delta_pip
from pyspark.sql import DataFrame, SparkSession, functions as F

In [28]:
# Load environment variables
load_dotenv()

api_key = os.getenv("API_KEY")
encryption_key = os.getenv("ENCRYPTION_KEY")

### BambooHR API Client

In [29]:
EMPLOYEES_REPORT_PAYLOAD = {
    "title": "custom-employees",
    "fields": [
        "displayName",
        "firstName",
        "lastName",
        "preferredName",
        "dateOfBirth",
        "maritalStatus",
        "ssn",
        "gender",
        "pronouns",
        "country",
        "city",
        "zipcode",
        "address1",
        "address2",
        "employee_access",
        "customShirtsize",
        "customHobbies",
        "allergies",
        "dietaryRestrictions",
        "jobTitle",
        "hireDate",
        "originalHireDate",
        "employeeStatusDate",
        "employmentStatus",
        "employmentHistoryStatus",  # FTE
        "terminationDate",
        "location",
        "workPhone",
        "mobilePhone",
        "workEmail",
        "department",
        "division",
        "workPhoneExtension",
        "supervisor",
        "supervisorEid",
    ],
}

In [33]:
class BambooHRClient:
    """A client for the BambooHR API.
    This client handles authentication, session management, and API requests
    to the BambooHR API or a placeholder API for demonstration purposes.
    """

    DOMAIN = "muttclip"
    BASE_URL = f"https://api.bamboohr.com/api/gateway.php/{DOMAIN}/v1"
    ENDPOINTS = {
        "company_information": "/company_information",
        # Headcount
        "employees": "/reports/custom",
        "employees_changed": "/employees/changed",
        "account_info_fields": "/meta/fields",
        "account_info_tab_fields": "/meta/tables",
        "account_list_fields": "/meta/lists",
        "account_list_users": "/meta/users",
        # Time Off
        "time_off_types": "/meta/time_off/types",
        "time_off_policies": "/meta/time_off/policies",
        "time_off_requests": "/time_off/requests",
        "time_off_whos_out": "/time_off/whos_out",
        # Hiring
        "applicant_tracking_jobs": "/applicant_tracking/jobs",
        "applicant_tracking_applications": "/applicant_tracking/applications",
        "applicant_application_details": "/applicant_tracking/applications/{appId}",
        ##
        "custom_report_test": "/reports/101",
    }

    def __init__(self, api_key: str, days_offset: int = 1):
        """Initialize the BambooHRClient with an API key."""
        self.api_key = api_key
        self.days_offset = days_offset
        self.base_url = self.BASE_URL
        self.session = self._create_session()
        self.headers = {
            "Content-Type": "application/json",
            "accept": "application/json",
            "Authorization": self._encode_auth_header(api_key),
        }

    def _encode_auth_header(self, api_key: str) -> str:
        """Encode the API key in Base64 for the Authorization header."""
        auth_string = f"{api_key}:x"
        encoded_bytes = base64.b64encode(auth_string.encode("utf-8"))
        return f"Basic {encoded_bytes.decode('utf-8')}"

    def _build_query_string(self, params: dict) -> str:
        """Helper method to construct a query string from a dictionary."""
        return "&".join(f"{k}={v}" for k, v in params.items())

    def _create_session(self) -> requests.Session:
        """Create a session with a retry strategy for handling transient errors."""
        retry_strategy = Retry(
            total=5,
            backoff_factor=2,
            status_forcelist=[429, 503],
            allowed_methods=["GET", "POST"],
        )

        # Create an adapter with the retry strategy
        adapter = HTTPAdapter(max_retries=retry_strategy)

        # Create a session and mount the adapter
        session = requests.Session()
        session.mount("https://", adapter)

        return session

    def get(
        self, endpoint_key: str = None, endpoint_path: str = None, params: dict = None
    ) -> dict:
        """
        Fetch data from a specified endpoint using the endpoint key or a direct endpoint path.

        Args:
            endpoint_key (str, optional): The key for the desired endpoint (e.g., "posts"). Defaults to None.
            endpoint_path (str, optional): The direct path for the endpoint (e.g., "/employees/123"). Defaults to None.
            params (dict, optional): Query parameters to include in the request.

        Returns:
            dict: The JSON response from the API.

        Raises:
            ValueError: If neither `endpoint_key` nor `endpoint_path` is provided.
            requests.exceptions.RequestException: If the request fails.
        """
        if endpoint_key:
            endpoint = self.ENDPOINTS.get(endpoint_key)
            if not endpoint:
                raise ValueError(f"Invalid endpoint key: {endpoint_key}")
        elif endpoint_path:
            endpoint = endpoint_path
        else:
            raise ValueError(
                "You must provide either an endpoint_key or an endpoint_path."
            )

        url = f"{self.base_url}{endpoint}"
        response = self.session.get(url, headers=self.headers, params=params)
        try:
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed with status {response.status_code}: {e}")
            raise

        return response.json()

    def get_employees_changed(self, change_type: str = None) -> dict:
        """
        Fetch the list of employees changed since a given number of days ago.

        Args:
            change_type (str, optional): Type of change to filter for ("inserted", "updated", "deleted").
                If not provided, all change types will be included.

        Returns:
            dict: The JSON response from the API, containing the list of employees
                who have changed since the specified timestamp.
        """
        since = datetime.now(timezone.utc) - timedelta(days=self.days_offset)
        since = since.strftime("%Y-%m-%dT%H:%M:%SZ")

        params = {"since": since}
        if change_type:
            params["type"] = change_type

        return self.get("employees_changed", params=params)

    def get_time_off_request(self) -> dict:
        """
        Fetch time-off requests within a specified date range.

        Returns:
            dict: The JSON response from the API, containing the list of time-off
                requests within the specified date range.
        """
        start_dt = datetime.now(timezone.utc) - timedelta(days=self.days_offset)
        end_dt = datetime.now(timezone.utc)
        start_dt = start_dt.strftime("%Y-%m-%d")
        end_dt = end_dt.strftime("%Y-%m-%d")
        params = {
            "start": start_dt,
            "end": end_dt,
        }

        return self.get("time_off_requests", params=params)

    def get_appl_trk_apps(self) -> dict:
        """
        Fetches applicant tracking applications updated since a specified number of days ago.

        This method generates the `newSince` query parameter using the current UTC time
        minus the specified number of days. It then sends a GET request to the
        `applicant_tracking_applications` endpoint with the constructed parameter.

        Returns:
            dict: The API response containing applicant tracking applications.
        """
        new_since = datetime.now(timezone.utc) - timedelta(days=self.days_offset)
        new_since = new_since.strftime("%Y-%m-%d %H:%M:%S")
        params = {"newSince": new_since}

        return self.get("applicant_tracking_applications", params=params)

    def get_application_details(self) -> list:
        """
        Fetches detailed information for each application in the applicant tracking system.

        This method first retrieves a list of applications using the `get_appl_trk_apps` method,
        extracts their IDs, and then fetches detailed information for each application by calling
        the BambooHR API endpoint `/applicant_tracking/applications/{applicationId}`.

        Returns:
            list: A list of dictionaries, where each dictionary contains detailed data for a single application.

        Notes:
            - The method uses the `get_appl_trk_apps` method to fetch the initial list of applications.
            - The `get` method is used with `endpoint_path` to fetch details for each `applicationId`.

        Error Handling:
            - If an exception occurs during the detailed fetch for a specific application,
            the method logs the error and continues processing the remaining application IDs.
        """
        # Fetch the initial applications report and get the application IDs
        applications_report = self.get_appl_trk_apps()
        application_ids = [
            app["id"] for app in applications_report.get("applications", [])
        ]
        # Fetch detailed data for each application
        detailed_applications = []
        for app_id in application_ids:
            try:
                details = self.get(
                    endpoint_path=f"/applicant_tracking/applications/{app_id}"
                )
                detailed_applications.append(details)
            except Exception as e:
                print(f"Failed to fetch details for application ID {app_id}: {e}")

        return detailed_applications

    def post(
        self, endpoint_key: str, data: dict = None, query_params: dict = None
    ) -> dict:
        """
        Send data to a specified endpoint using the POST method.

        Args:
            endpoint_key (str): The key for the desired endpoint.
            data (dict, optional): The JSON payload to send in the POST request.
            query_params (dict, optional): Query parameters to include in the URL.

        Returns:
            dict: The JSON response from the API.

        Raises:
            ValueError: If an invalid endpoint key is provided.
            requests.exceptions.RequestException: If the request fails.
        """
        endpoint = self.ENDPOINTS.get(endpoint_key)
        if not endpoint:
            raise ValueError(f"Invalid endpoint key: {endpoint_key}")

        if query_params:
            query_string = self._build_query_string(query_params)
            url = f"{self.base_url}{endpoint}?{query_string}"
        else:
            url = f"{self.base_url}{endpoint}"

        response = self.session.post(url, headers=self.headers, json=data)
        try:
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Request failed with status {response.status_code}: {e}")
            raise

        return response.json()

    def create_employees_report(self) -> dict:
        """
        Generate a custom report of employees and all their relevant data.

        Returns:
            dict: The JSON response from the API with the generated report.
        """
        payload = EMPLOYEES_REPORT_PAYLOAD
        q_params = {"format": "json", "onlyCurrent": "true"}

        return self.post("employees", data=payload, query_params=q_params)

In [34]:
# Initialize the client
client = BambooHRClient(api_key=api_key, days_offset=30)

# Create a data directory to store the JSON outputs
os.makedirs("data/api_response", exist_ok=True)


def save_json_data(data: dict, filename: str) -> None:
    """Save JSON data to a file."""
    with open(f"data/api_response/{filename}", "w") as f:
        json.dump(data, f, indent=4)
    print(f"{filename} file saved successfully")

In [ ]:
# Map report methods or endpoint keys to filenames
report_mappings = {
    "company_information": "company_information.json",
    "employees": lambda: client.create_employees_report(),
    "employees_changed": lambda: client.get_employees_changed(),
    "account_info_fields": "account_info_fields.json",
    "account_info_tab_fields": "account_info_tab_fields.json",
    "account_list_fields": "account_list_fields.json",
    "account_list_users": "account_list_users.json",
    "time_off_types": "time_off_types.json",
    "time_off_policies": "time_off_policies.json",
    "time_off_requests": lambda: client.get_time_off_request(),
    "time_off_whos_out": "time_off_whos_out.json",
    "applicant_tracking_jobs": "applicant_tracking_jobs.json",
    "applicant_tracking_applications": lambda: client.get_appl_trk_apps(),
    "applicant_application_details": lambda: client.get_application_details(),
    # "custom_report_test": "custom_report_test.json",
}

# Fetch and save reports dynamically
for report_key, filename_or_callable in report_mappings.items():
    if callable(filename_or_callable):
        data = filename_or_callable()
        filename = f"{report_key}.json"
    else:
        data = client.get(endpoint_key=report_key)
        filename = filename_or_callable
    save_json_data(data, filename)

### Transform and Save data in delta format

In [25]:
class DeltaFileManager:
    """Handles the creation, transformation, and storage of data in Delta format."""

    def __init__(self, app_name: str = "MyApp", encryption_key: str = None):
        """
        Initializes the Spark session with Delta Lake configurations and optionally sets the encryption key.

        Args:
            app_name (str): Name of the Spark application.
            encryption_key (str): Encryption key for AES encryption (default: None).
        """
        builder = (
            SparkSession.builder.appName(app_name)
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config(
                "spark.sql.catalog.spark_catalog",
                "org.apache.spark.sql.delta.catalog.DeltaCatalog",
            )
        )
        self.spark = configure_spark_with_delta_pip(builder).getOrCreate()
        self.encryption_key = encryption_key

    def convert_to_json_string(self, data: Union[List[dict], dict]) -> list:
        """
        Converts a dictionary or a list of dictionaries into a list of JSON strings.

        Args:
            data (Union[List[dict], dict]): The input data, either a list of dictionaries
                or a single dictionary.

        Returns:
            list: A list of JSON strings.

        Raises:
            ValueError: If the input data is neither a list of dictionaries nor a dictionary.
        """
        if isinstance(data, dict):
            # Convert a single dictionary to a JSON string
            return [json.dumps(data, default=str)]
        elif isinstance(data, list) and all(isinstance(item, dict) for item in data):
            # Convert each dictionary in the list to a JSON string
            return [json.dumps(record, default=str) for record in data]
        else:
            raise ValueError(
                "Input data must be a dictionary or a list of dictionaries."
            )

    def create_schemaless_df(self, json_strings: list) -> DataFrame:
        """Converts a list of JSON strings into a schemaless DataFrame."""
        return self.spark.createDataFrame(json_strings, StringType())

    def add_processed_dt(self, df: DataFrame) -> DataFrame:
        """Adds a processed timestamp column to a DataFrame."""
        return df.withColumn("processed_at", F.current_timestamp())

    def encrypt_columns(
        self,
        df: DataFrame,
        columns: list,
        encryption_mode: str = "ECB",
    ) -> DataFrame:
        """
        Encrypts the specified columns in the DataFrame using AES encryption and Base64 encoding.

        Args:
            df (DataFrame): The input DataFrame.
            columns (list): List of column names to encrypt.
            encryption_mode (str): The encryption mode for AES (default: "ECB").

        Returns:
            DataFrame: The DataFrame with encrypted columns.
        """
        if not self.encryption_key:
            raise ValueError(
                "Encryption key is not set. Please provide an encryption key."
            )

        for col_name in columns:
            encrypted_col = F.expr(
                f"aes_encrypt({col_name}, '{self.encryption_key}', '{encryption_mode}')"
            )
            base64_encoded_col = F.base64(encrypted_col)
            df = df.withColumn(col_name, base64_encoded_col)
        return df

    def save_to_delta(
        self, df: DataFrame, path: str, repartition: int = 1, mode: str = "append"
    ):
        """Saves a DataFrame to a Delta table."""
        df.repartition(repartition).write.format("delta").mode(mode).save(path)

In [ ]:
# Initialize the manager
manager = DeltaFileManager(encryption_key=encryption_key)

# Directory containing JSON files
data_folder = "data/api_response/"

# Iterate over all JSON files in the folder
for filename in os.listdir(data_folder):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(data_folder, filename)
        # Read the JSON file
        with open(file_path, "r") as file:
            data = json.load(file)
        # Convert the JSON data to JSON strings
        json_strings = manager.convert_to_json_string(data)
        # Create schemaless DataFrame
        df = manager.create_schemaless_df(json_strings)
        # Add a processed timestamp column
        df = manager.add_processed_dt(df)
        # Encrypt data (if required)
        df = manager.encrypt_columns(df, columns=["value"])
        # Generate a Delta table path based on the filename
        table_name = filename.replace(".json", "")
        delta_table_path = f"data/delta_tables/{table_name}"
        # Save the DataFrame to Delta table
        manager.save_to_delta(df, delta_table_path, mode="overwrite")

        print(f"Processed and saved: {table_name}")

### Decrypt value

In [18]:
def decrypt_columns(
    df: DataFrame,
    columns: list,
    encryption_key: str = encryption_key,
    encryption_mode: str = "ECB",
) -> DataFrame:
    """
    Decrypts the specified columns in the DataFrame using AES decryption and Base64 decoding.

    Args:
        df (DataFrame): The input DataFrame.
        columns (list): List of column names to decrypt.
        encryption_key (str): The decryption key for AES decryption.
        encryption_mode (str): The decryption mode for AES (default: "ECB").

    Returns:
        DataFrame: The DataFrame with decrypted columns.
    """
    for col_name in columns:
        decrypted_col = F.expr(
            f"aes_decrypt(unbase64({col_name}), '{encryption_key}', '{encryption_mode}')"
        ).cast("string")
        df = df.withColumn(col_name, decrypted_col)

    return df

In [ ]:
# Check decryption
builder = (
    SparkSession.builder.appName("app_name")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

delta_table_path = "data/delta_tables/employees"

df = spark.read.format("delta").load(delta_table_path)
df = decrypt_columns(df, columns=["value"])
df.show(5, truncate=False)